In [1]:
import numpy as np

# Exception

프로그래밍 중 잘못된 문법을 입력하면 에러 메시지를 만나게 된다. 

이를 exception이 발생했다고도 한다.  

Python의 대부분의 exception들은 ```Exception```이라는 class를 상속하여 만들어진다.

```IndexError```, ```NameError```, ```KeyError```, ```ValueError```, ```SyntaxError```, ```IOError``` 등이 있다.

Exception을 발생하기 위해서는 ```raise``` 키워드를 사용한다.

예를 들어, 다음과 같이 사용한다.

```if badcondtion: raise Exception("Someting wrong")  ```

In [2]:
def factorial(n):
    if not (isinstance(n, (int))):
        raise TypeError("An integer is expected")
    if not (n >=0): 
        raise ValueError("A positive number is expected")
    
    if n == 0: return 1
    else: return n*factorial(n-1)

위 함수는 ```factorial(1.1)``` 혹은 ```factorial(-12)```와 같이 활용하면 에러가 발생할 것이다.

In [3]:
factorial(5)

120

만약 에러가 발생했을 때, 프로그램을 멈추지 않고, 모종의 방법으로 에러를 처리하고 넘어가고 싶다면, ```try``` 구문이 이용된다.  

아래 코드에서 ```try``` 블록의 ```factorial(n)```는 원래는 에러가 발생하는 구문이지만, ```try``` 블록 안에 있을 경우, 에러 메시지가 발생하지 않는다.  

만약 ```try``` 블록 내의 코드를 실행하다가 Exception이 발생하면 ```except```로 넘어가 그 블록의 코드를 실행한다.

In [4]:
n = -3
try:
    print(factorial(n))
except ValueError:
    print(factorial(-n))

6


In [5]:
n = 4.1
try:
    print(factorial(n))
except TypeError:
    print(factorial(int(n)))
except ValueError:
    print(factorial(-n))

24


여러 exception들을 동일한 방법으로 처리하고 싶을 때, 다음과 같은 문법이 사용될 수 있다.   
```except (RuntimeError, ValueEroor, IOError)```

각 exception들을 다르게 처리하고자 하면, 여러 ```except```를 사용한다.  

In [6]:
try: 
    f = open('data.txt', 'r') 
    data = f.readline() 
    value = float(data) 
except FileNotFoundError as FnF: 
    print(f' {FnF.strerror}: {FnF.filename}') 
except ValueError: 
    print("Could not convert data to float.")

 No such file or directory: data.txt


```try``` 구문은 ```else```와 ```finally``` 구문과 같이 사용할 수 있다.

```else``` 절은 exception이 발생하지 않아 ```except``` 절을 실행하지 않았을 때 실행된다.

```finally``` 블록은 ```try``` 블록에서 오류가 발생하는지 여부에 관계없이 항상 실행된다.

### user defined exception

Python의 built-in exception을 이용할 수도 있지만, 경우에 따라 직접 작성한 exception을 이용할 수도 있다.

```Exception``` 클래스를 상속하도록 한다.

User defined exception의 이름은 ```Error```로 끝나도록 하는 것을 추천한다.

In [7]:
class MyError(Exception):    #Exception`클래스를 상속
    def __init__(self, expr):
        self.expr = expr
    def __str__(self):
        return str(self.expr)

In [8]:
try:
    x = np.random.rand()
    if x < 0.5: 
        raise MyError(x)
except MyError as e:
    print("Random number too small", e.expr)
else:
    print(x)

Random number too small 0.4354613307868179


```with``` 구문은 excecptin을 함축하고 있다.  
Python에서 file을 열어 작업할 때 종종 다음의 구문을 이용한다. 

```
with open('data.txt', 'w') as f:
    process_file_data(f)
```

이 구문은 다음의 exception 구문과 동일하다.

```
f = open('data.txt', 'w')
try:
    process_file_data(f)
execpt:
    ...
finally:
    f.close()
```

```with```를 이용한 코드에서 ```f```는 context manager의 역할을 담당한다.  
Python에서 context manager는 ```__enter__```와 ```__exit__``` method가 구현된 클래스로 정의된다.  
```__enter__```는 preprocessing을 담당하고, ```__exit__```은 postprocessing을 담당한다.  

Context manager의 또다른 예로서 ```numpy.errstate```가 있다.  
이것은 부동 소수점 오류 처리를위한 컨텍스트 관리자이다.  

In [9]:
import numpy as np  
np.errstate.__enter__

<function numpy.core._ufunc_config.errstate.__enter__(self)>

In [10]:
np.errstate.__exit__

<function numpy.core._ufunc_config.errstate.__exit__(self, *exc_info)>

```with``` 구문과 같이 사용함으로써 부동 소수점 오류가 발생했을 때, 할 일을 다르게 명시할 수 있다.

In [11]:
# 오류를 무시하기
with np.errstate(invalid='ignore'):
    print(np.sqrt(-1)) # prints 'nan'

nan


In [12]:
# 오류를 발생시키지는 않으나, 경고 메시지 띄우기
with np.errstate(invalid='warn'):
    print(np.sqrt(-1)) # prints 'nan' and # 'RuntimeWarning: invalid value encountered in sqrt'

nan


C:\Users\Owner\AppData\Local\Temp\ipykernel_43152\4056108474.py:3: RuntimeWarning: invalid value encountered in sqrt
  print(np.sqrt(-1)) # prints 'nan' and # 'RuntimeWarning: invalid value encountered in sqrt'


다음의 코드를 실행하면 에러가 발생한다.

```
with errstate(invalid='raise'):
    print(np.sqrt(-1)) # prints nothing and raises FloatingPointError
```

### Finding errors

Exception이 발생했을 때, 해석하는 방법을 좀 더 자세히 살펴본다. 

Exception이 발생하면, 에러 메시지에서 call stack을 보게될 것이다.  
Call stack은 exception이 발생한 코드를 호출하는 함수들을 추적하여 보여준다.

```
def f():
    g()
def g():
    h()
def h():
    1//0

f()


---------------------------------------------------------------------------
ZeroDivisionError                         Traceback (most recent call last)
Input In [53], in <cell line: 8>()
      5 def h():
      6     1//0
----> 8 f()

Input In [53], in f()
      1 def f():
----> 2     g()

Input In [53], in g()
      3 def g():
----> 4     h()

Input In [53], in h()
      5 def h():
----> 6     1//0

ZeroDivisionError: integer division or modulo by zero
```

다른 예제를 보자.

```
def f(a):
    g(a)
def g(a):
    h(a)
def h(a):
    raise Exception(f'An exception just to provoke a strack trace and a value a={a}')

f(23)


---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Input In [51], in <cell line: 8>()
      5 def h(a):
      6     raise Exception(f'An exception just to provoke a strack trace and a value a={a}')
----> 8 f(23)

Input In [51], in f(a)
      1 def f(a):
----> 2     g(a)

Input In [51], in g(a)
      3 def g(a):
----> 4     h(a)

Input In [51], in h(a)
      5 def h(a):
----> 6     raise Exception(f'An exception just to provoke a strack trace and a value a={a}')

Exception: An exception just to provoke a strack trace and a value a=23
```

Python에는 built-in debugger인 ```pdb```가 있다.   

코드 실행 중 중단하여 디버거 세션을 열고 싶은 곳에 ```pdb.set_trace()``` 코드를 넣는다.

디버깅 모드로 진입하면, (Pdb) 프롬프트가 나오게 되고, 여기서 여러 PDB 명령을 사용할 수 있다. 

아래는 디버깅 모드에서 사용할 수 있는 간단한 command list이다.


| Command | Action |
|:----|:----|
| h | Help |
| l | 현재 위치의 소스 코드 출력 |
| q | 끝내기 |
| c | 계속 실행 |
| r | 현재 함수의 return 직적까지 수행 |
| n | 다음 라인으로 이동 |
| p expression | expression을 계산하여 현재 값을 출력 |

In [15]:
import pdb

def complex_to_polar(z):
    pdb.set_trace() 
    r = np.sqrt(z.real ** 2 + z.imag **2)
    phi = np.arctan2(z.imag, z.real)
    return (r, phi)

z = 3 + 5j
r, phi = complex_to_polar(z)

> c:\users\owner\appdata\local\temp\ipykernel_43152\1011003366.py(5)complex_to_polar()

ipdb> c
